# Marketing‑Mix Modeling (MMM) – With Ad‑Stock & Saturation
This script extends the “plain” MMM example by introducing **carry‑over(ad‑stock) effects** and **diminishing‑return curves** for paid media.

The workflow is:
1. Load weekly sales + media data.
2. Generate Fourier seasonality terms.
3. Tell the optimizer which media columns require ad‑stock.
4. Run Optuna to tune ARIMAX *plus* per‑channel carry‑over & curve
   parameters (decay rate, saturation strength, etc.).
5. Fit the final model, visualise the estimated ad‑stock curves, and
   save everything (study, model, predictions).
6. Translate coefficients into yen **contribution**, compute **ROI**,
   and plot Contribution × ROI for budget insight.

Every block is annotated so you can follow the logic step‑by‑step.

In [1]:
# ────────────────────────────────────────────────────────────────────
# STEP 1. Imports
# ────────────────────────────────────────────────────────────────────
# `mmm_functions.py` bundles helper utilities:
#   • Fourier generator
#   • Optuna objective that supports carry‑over & saturation
#   • Contribution / ROI calculators & plotting helpers

from mmm_functions import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib   # for persisting Optuna study & model objects

# ────────────────────────────────────────────────────────────────────
# STEP 2. Load Weekly Data
# ────────────────────────────────────────────────────────────────────
DATASET_PATH = "df_mmm.csv"
df = pd.read_csv(
    DATASET_PATH,
    parse_dates=["week"],
    index_col="week",
)

# ────────────────────────────────────────────────────────────────────
# STEP 3. Declare Media Columns that Need Ad‑Stock
# ────────────────────────────────────────────────────────────────────
# In MMM jargon, “apply_effects_features” are *paid media drivers*
# whose influence is assumed to *linger* beyond the week they were
# spent (carry‑over) and to *saturate* at high spend levels.
apply_effects_features = ["traditional", "internet", "promotional"]

# ────────────────────────────────────────────────────────────────────
# STEP 4. Add Fourier Seasonality Terms
# ────────────────────────────────────────────────────────────────────
# 5 sine + 5 cosine pairs (10 columns) for a 52‑week period.
df = add_fourier_terms(df, num=5, seasonal=52.25)

# ────────────────────────────────────────────────────────────────────
# STEP 5. Restrict to Last ≈ 5 Years
# ────────────────────────────────────────────────────────────────────
data_term = int(52.25 * 5)

X = df.drop(columns=['sales'])[-data_term:]
y = df['sales'][-data_term:]

In [ ]:
# ────────────────────────────────────────────────────────────────────
# STEP 6. Hyper‑parameter Search with Ad‑Stock
# ────────────────────────────────────────────────────────────────────
# `regarima_objective` optimises:
#   • AR / MA orders, BoxCox/log transform flags
#   • Each channel’s carry‑over decay α (0–1, where 1 = no decay)
#   • Each channel’s saturation “shape” (e.g. Hill or power curve)
#   • Regularisation strength (ridge / lasso)
#
# n_trials=100 keeps runtime reasonable for demo purposes; increase
# to 500+ for better convergence.
print("Running Optuna with ad‑stock tuning ...")
study = run_optimization(
    regarima_objective, 
    X, y, 
    apply_effects_features, 
    n_trials=1000)

# Persist the Optuna study so you can inspect trials later
joblib.dump(study, "ridgeMMM_study.joblib")

print("Best validation metric:", study.best_value)
print("Best parameters:")
for k, v in study.best_trial.params.items():
    print(f"  • {k}: {v}")
print()



Running Optuna with ad‑stock tuning ...


  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:

# ────────────────────────────────────────────────────────────────────
# STEP 7. Fit Final Model on Full Data Window
# ────────────────────────────────────────────────────────────────────
trained_model, model_params, pred = create_model_from_trial_regarima(
    study.best_trial,
    X,
    y,
    apply_effects_features,
)

# `model_params` returns a tuple:
#   • [0] carry‑over parameters (decay α for each channel)
#   • [1] curve parameters       (saturation coefficient, etc.)
best_carryover_params = model_params[0]
best_curve_params     = model_params[1]

# Visualise the estimated ad‑stock and saturation curves
plot_effects(
    best_carryover_params,
    best_curve_params,
    apply_effects_features,
)

# Save trained model to disk for later scoring
joblib.dump(trained_model, "ridgeMMM_trained_model.joblib")


In [ ]:
# ────────────────────────────────────────────────────────────────────
# STEP 8. Save Fitted Predictions (optional)
# ────────────────────────────────────────────────────────────────────
pred = trained_model.predict(X)
pred = pd.DataFrame(pred, index=X.index, columns=['y'])
pred.to_csv('pred.csv')

In [ ]:
# ────────────────────────────────────────────────────────────────────
# STEP 9. Calculate Weekly Contribution
# ────────────────────────────────────────────────────────────────────
contribution = calculate_and_plot_contribution(
    y, X, 
    trained_model, 
    (0, np.max(y)*1.1), 
    apply_effects_features
)
print("Weekly contribution head:")
print(contribution.head(), "\n")

In [ ]:
# ────────────────────────────────────────────────────────────────────
# STEP 10. Summarise Contribution Share
# ────────────────────────────────────────────────────────────────────
contribution_totals = summarize_and_plot_contribution(contribution)
print("Total contribution & share:")
print(contribution_totals, "\n")

In [ ]:
# ────────────────────────────────────────────────────────────────────
# STEP 11. Calculate Marketing ROI
# ────────────────────────────────────────────────────────────────────
# マーケティングROIの算出
ROI = calculate_marketing_roi(
    X[apply_effects_features], 
    contribution[apply_effects_features]
)

print(ROI)


In [ ]:
# ────────────────────────────────────────────────────────────────────
# STEP 12. Contribution × ROI Scatter Plot
# ────────────────────────────────────────────────────────────────────
data_to_plot = plot_scatter_of_contribution_and_roi(
    X[apply_effects_features], 
    contribution[apply_effects_features]
)

print(data_to_plot)